In [1]:
import pandas as pd  # pandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# a akos-0_label_counts.ipynb altal előállított label_categories_with_counts_2.csv-t hasznalva train set-et csinál
# a max_label mondja meg, hogy az első X label-t tegye bele a train-be
# 
max_label = 80
output_dataset_name = "../datasets/train1_80labels_withdaytimes.csv"

# Adatbázis séma (Diagram)

<img src="table_diagram.png">

# Adattáblák

## events

In [3]:
events = pd.read_csv("../datasets/events.csv", sep=",")

In [4]:
events.head()

,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


In [5]:
events.count()

event_id     3252950
device_id    3252950
timestamp    3252950
longitude    3252950
latitude     3252950
dtype: int64

## gender_age_train

In [6]:
gender_age_train = pd.read_csv("../datasets/gender_age_train.csv", sep=",")

In [7]:
gender_age_train.head()

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [8]:
gender_age_train.count()

device_id    74645
gender       74645
age          74645
group        74645
dtype: int64

In [9]:
gender_age_train.groupby('group')['device_id'].count()

group
F23-      5050
F24-26    4190
F27-28    3118
F29-32    4628
F33-42    5561
F43+      4194
M22-      7488
M23-26    9605
M27-28    5445
M29-31    7309
M32-38    9476
M39+      8581
Name: device_id, dtype: int64

In [10]:
gender_age_train.groupby('gender')['device_id'].count()

gender
F    26741
M    47904
Name: device_id, dtype: int64

## phone_brand_device_model

In [11]:
phone_brand_device_model = pd.read_csv("../datasets/phone_brand_device_model.csv", sep=",")

In [12]:
phone_brand_device_model.head()

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


In [13]:
# a kínai márkákat lefordítjuk...

In [14]:
translate = pd.read_csv("../datasets/translate.csv", sep=",")

In [15]:
translate.head()

,phone_brand,phone_brand_eng
0,三星,samsung
1,天语,Ktouch
2,海信,hisense
3,联想,lenovo
4,欧比,obi


In [16]:
phone_brand_device_model= phone_brand_device_model.merge(translate,on='phone_brand',how='right')

In [17]:
phone_brand_device_model.head()

,device_id,phone_brand,device_model,phone_brand_eng
0,-8890648629457979026,小米,红米,Xiaomi
1,1277779817574759137,小米,MI 2,Xiaomi
2,-3883532755183027260,小米,MI 2S,Xiaomi
3,-5827952925479472594,小米,MI One Plus,Xiaomi
4,-5776341595519925628,小米,MI 2A,Xiaomi


In [18]:
# ha tobb van, akkor csak az elsot hagyjuk meg...
phone_brand_device_model = phone_brand_device_model.drop_duplicates('device_id',keep='first').set_index('device_id')

## app_events

In [19]:
app_events = pd.read_csv("../datasets/app_events.csv", sep=",")

In [20]:
app_events.head()

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


In [21]:
app_events.count()

event_id        32473067
app_id          32473067
is_installed    32473067
is_active       32473067
dtype: int64

In [22]:
app_events.groupby('event_id')['app_id'].agg('count').count()

1488096

In [23]:
# osszesen 1488096 db esemény van, ahhoz van 32473067 alkalmazás esemény 

## app_labels

In [24]:
app_labels = pd.read_csv("../datasets/app_labels.csv", sep=",")

In [25]:
app_labels.head()

,app_id,label_id
0,7324884708820027918,251
1,-4494216993218550286,251
2,6058196446775239644,406
3,6058196446775239644,407
4,8694625920731541625,406


In [26]:
app_labels.count()

app_id      459943
label_id    459943
dtype: int64

## label_categories

In [27]:
label_categories = pd.read_csv("../datasets/label_categories.csv", sep=",")

In [28]:
label_categories.head()

,label_id,category
0,1,NaN
1,2,game-game type
2,3,game-Game themes
3,4,game-Art Style
4,5,game-Leisure time


In [29]:
label_categories.count()

label_id    930
category    927
dtype: int64

In [30]:
# itt van 3 sor NaN. Igazából szerintem nem számít.

# Adatok előkészítése

Szerintem feladatok:
    - a gender_age_train össze kellene join-olni a phone_brand_device_modellel
    - ugyancsak a gender_age_train-t össze kell join-olni az events-app_events-app_labels -en keresztül a label_categories-vel
    - a label kategória dummy változó lenne. A problémám, hogy itt akkro  930 db lenne...
    - bemenő változók: márka, alkalmazás label dummy. Célváltozó: group


## Label counts

itt külön notebookban megszámoltam és elmentettem az egyes label-ek számosságát

In [31]:
label_categories = pd.read_csv("../datasets/label_categories_with_counts_2.csv", sep=",")

In [32]:
label_categories = label_categories.sort_values("all_cou", ascending=0)

In [33]:
label_categories["label_id_use"] = label_categories["label_id"] 

In [34]:
label_array = []

In [35]:
for a in range(0,max_label):
    label_array.append("label_%d"%a)


In [36]:
for a in range(max_label, len(label_categories)):
    label_categories.at[a, 'label_id_use'] = 22222

In [37]:
# A label_id_use-ban az elso max_label helyen ott van a tenyleges id, a tobbi esetben egy gyujto: 22222

In [38]:
label_categories.head()

,Unnamed: 0,label_id,category,not_active_cou,active_cou,all_cou,label_id_use
0,489,548,Industry tag,14394096.0,9846501.0,24240597.0,548
1,629,704,Property Industry 2.0,10017203.0,6807461.0,16824664.0,704
2,490,549,Property Industry 1.0,7567242.0,6113200.0,13680442.0,549
3,359,405,Custom label,5902239.0,2337647.0,8239886.0,405
4,638,713,Services 1,5084162.0,3098384.0,8182546.0,713


# app_label merge

In [39]:
# az app_labels-be beleteszuk a label_id_use-t
app_labels = app_labels.merge(label_categories, on="label_id", how="left")

In [40]:
app_labels.head()

,app_id,label_id,Unnamed: 0,category,not_active_cou,active_cou,all_cou,label_id_use
0,7324884708820027918,251,232,Finance,3518798.0,1484275.0,5003073.0,251
1,-4494216993218550286,251,232,Finance,3518798.0,1484275.0,5003073.0,251
2,6058196446775239644,406,360,unknown,523546.0,204958.0,728504.0,406
3,6058196446775239644,407,361,DS_P2P net loan,523546.0,204958.0,728504.0,407
4,8694625920731541625,406,360,unknown,523546.0,204958.0,728504.0,406


In [41]:
# a label_id_use legyen a label_id
app_labels.rename(columns={"label_id": "label_id_original", "label_id_use":"label_id"}, inplace=True)
app_labels.head()


,app_id,label_id_original,Unnamed: 0,category,not_active_cou,active_cou,all_cou,label_id
0,7324884708820027918,251,232,Finance,3518798.0,1484275.0,5003073.0,251
1,-4494216993218550286,251,232,Finance,3518798.0,1484275.0,5003073.0,251
2,6058196446775239644,406,360,unknown,523546.0,204958.0,728504.0,406
3,6058196446775239644,407,361,DS_P2P net loan,523546.0,204958.0,728504.0,407
4,8694625920731541625,406,360,unknown,523546.0,204958.0,728504.0,406


In [42]:
events.head()

,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


In [43]:
app_events.head()

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


In [44]:
#stop

## Tanító1 -  esemény szám + label számosság

egy olyan df-et hozunk létre, ahol minden device-hez ott van, hogy milyen típusú alkalmazás label-ek vannak, milyen a tipusu keszuleke van es hany esemenye.
Magyarázó oszlopok:
- events_num : mennyi esemenye volt
- lavel_x : az alkalmazasok (aktiv + installed kulon ) label-jei
- phone_brand
- device_model
- events_per_day :events_num / (max_timestamp - min timestamp) * 60*60*24  - eseményszám osztva a napok szamaval
- min_timestamp : mikor volt az első eseménye
- max_timestamp : mikor volt az utolsó eseménye
- weekday/weekend_morning/daytime/evening/night - események szerint

In [45]:
app_events.head()

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


In [46]:
def checkIfInDOWANdHours(t, dow=[5,6], hours=[22,23,0,1,2,3,4,5]):
    if pd.to_datetime(t).weekday() not in dow:
        return 0
    if pd.to_datetime(t).hour not in hours:
        return 0
    return 1

In [47]:
def getByDeviceID(device_id):
    df = events[ events["device_id"] == device_id ]
    df2 = df.merge(app_events , on="event_id", how="inner")
    df2 = df2.merge(app_labels, on="app_id", how="left")
    # 0=Hétfő, 1 = Kedd, 2-Szerda, 3-Csüt, 4-Péntek, 5-Szombat, 6-Vas, 
    df["weekday_morning"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [0,1,2,3,4], [6,7,8]))
    df["weekday_daytime"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [0,1,2,3,4], [9,10,11,12,13,14,15,16,17]))
    df["weekday_evening"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [0,1,2,3,4], [18,19,20,21]))
    df["weekday_night1"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [0,1,2,3], [22,23,0,1,2,3,4,5]))
    df["weekday_night2"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [4], [0,1,2,3,4,5]))
    df["weekday_night"] = df["weekday_night2"] + df["weekday_night1"] 
    df["weekend_morning"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [5,6], [6,7,8]))
    df["weekend_daytime"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [5,6], [9,10,11,12,13,14,15,16,17]))
    df["weekend_evening"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [5,6], [18,19,20,21]))
    df["weekend_night1"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [5], [22,23,0,1,2,3,4,5]))    
    df["weekend_night2"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [6], [22,23]))
    df["weekend_night3"] = df['timestamp'].apply(lambda x: checkIfInDOWANdHours(x, [4], [22,23]))
    df["weekend_night"] = df["weekend_night1"] + df["weekend_night2"] + df["weekend_night3"]
    
    return df, df2

In [48]:
def calcRow1(device_id):
    events1, df = getByDeviceID(device_id)
    ret = {}
    ret["device_id"] = [device_id]
    # esemenyek szama
    ret["events_num"] = df.groupby("event_id").agg("count").count().device_id 
    ret["min_timestamp"] = df['timestamp'].min()
    ret["max_timestamp"] = df['timestamp'].max()
    ret["events_per_day"] = pd.Timedelta(pd.to_datetime(ret["max_timestamp"]) - pd.to_datetime(ret["min_timestamp"])).seconds / ret["events_num"] / 60 / 60 / 24
    # label_id szamolasok
    for r in label_categories["label_id_use"].unique():
        colname = "label_%d"%r
        ret[colname] = [df[(df["label_id"] == r) & (df["is_active"] == 1)].count()["device_id"]]
        colname = "label_i_%d"%r
        ret[colname] = [df[(df["label_id"] == r) & (df["is_active"] == 0)].count()["device_id"]]
    for x in ["weekday_morning", "weekday_daytime", "weekday_evening", "weekday_night", "weekend_morning", "weekend_daytime", "weekend_evening", "weekend_night" ]:
        ret[x] = events1[x].sum()

    return pd.DataFrame(data=ret)

In [49]:
import warnings
warnings.filterwarnings('ignore')


In [50]:
calcRow1(6352067998666467520)
#calcRow1(520)


,device_id,events_num,min_timestamp,max_timestamp,events_per_day,label_548,label_i_548,label_704,label_i_704,label_549,...,label_22222,label_i_22222,weekday_morning,weekday_daytime,weekday_evening,weekday_night,weekend_morning,weekend_daytime,weekend_evening,weekend_night
0,6352067998666467520,11,2016-05-03 22:14:30,2016-05-05 17:27:58,0.07282,70,34,57,19,39,...,104,40,0,8,1,2,0,0,0,0


In [51]:
# sorra megyunk a device_id-kon. Eleg lassu lesz
max = 100000
df_train = None
for r in gender_age_train["device_id"].unique():
    #print("Start", r)
    x = calcRow1(r)
    if df_train is None:
        df_train = x
    else:
        df_train = df_train.append(x)
    max = max -1 
    if max < 0 :
        break
df_train.head()
df_train = df_train.merge(gender_age_train, on="device_id", how="inner")
df_train = df_train.merge(phone_brand_device_model, on="device_id", how="left")
df_train.to_csv(output_dataset_name)

KeyboardInterrupt: 

In [ ]:
df_train.head()